<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-27 09:43:57
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.37 C
-------------------
Today PnL: -74.77 K (-0.54%)
Current PnL: -37.00 L (-23.13%)
CY Booked + Current PnL: -21.32 L (-13.33%)
-------------------
Total profit:  1.37 L
Total loss:  -38.37 L
-------------------
Total Booked + Current PnL: 5.66 L (4.25%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.41%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.04 C (75.39%)
Deployed:  1.33 C
Current:  1.37 C
CAGR/XIRR %: 1.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.00,5.53,11.50,17.67,18260.0,8326.0,158786.0,478.51,27.69,55.0,L-LC,11.02,185.0,0.46,1.18,27.47,XY25,ATH,MINING
50,MASFIN,-1.27,2.45,18.66,21.57,18731.0,2400.0,100380.0,397.04,-11.47,54.0,H-SC,6.73,167.0,0.13,0.74,46.48,XR,ATH,FINANCE
77,TTKPRESTIG,-0.82,-32.03,47.26,0.09,32373.0,-32277.0,68500.0,770.00,59.02,31.0,M-SC,11.61,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
37,ICICIGI,-0.94,2.14,16.84,19.34,34419.0,4280.0,204389.0,2252.93,-14.19,53.0,X-MC,1.07,66.0,0.12,1.51,18.42,X40,ATH,INSURANCE
51,MEDANTA,-1.23,-4.20,29.51,24.08,35891.0,-5326.0,121624.0,1486.00,-4.88,51.0,X-SC,7.96,87.0,-0.15,0.90,11.40,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BSOFT,2.02,-29.02,112.44,50.79,112251.0,-40821.0,99832.0,831.70,-8.77,45.0,H-SC,5.26,172.0,-0.36,0.74,16.24,XR,ATH,IT
49,LAOPALA,1.54,-37.94,124.11,39.09,100985.0,-49738.0,81367.0,464.00,77.89,58.0,H-SC,13.40,154.0,-0.49,0.60,8.81,AR,NTT,CERAMICS
24,DIXON,1.37,-28.90,84.61,31.26,138828.0,-66688.0,164080.0,18931.72,-68.22,35.0,X-MC,24.19,54.0,-0.48,1.22,1.37,X40N,ATH,IT
43,INFY,1.27,-11.13,51.05,34.24,141956.0,-34816.0,278072.0,1972.00,-29.97,26.0,X-LC,8.69,3.0,-0.25,2.06,2.35,X40,NTT,IT
69,SURYODAY,1.22,-27.46,69.60,23.03,90415.0,-49164.0,129907.0,216.00,42.54,47.0,H-SC,5.65,166.0,-0.54,0.96,28.89,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,-3.69,-13.78,123.18,92.42,295839.0,-38391.0,240168.0,400.00,106.18,67.0,H-SC,8.04,174.0,-0.13,1.78,32.30,XY24,NTT,BANKS
17,CAMS,-2.81,-9.76,37.94,24.48,86227.0,-24585.0,227271.0,950.00,-13.46,37.0,X-SC,5.62,89.0,-0.29,1.68,11.67,X40N,NTT,MISC
5,ANGELONE,-2.80,-90.19,1165.86,24.20,349175.0,-275300.0,29950.0,3033.00,-85.87,38.0,X-SC,2.50,97.0,-0.79,0.22,25.43,X40N,NTT,FINANCE
55,RAJESHEXPO,-1.98,-71.41,250.00,0.07,98050.0,-97957.0,39220.0,518.00,1336.87,32.0,L-SC,9.86,268.0,-1.00,0.29,0.00,OX40N,NTT,JEWELLERY
71,TANLA,-1.76,-47.93,321.90,119.69,453847.0,-129771.0,140990.0,1943.92,-47.43,44.0,H-SC,20.27,147.0,-0.29,1.04,11.38,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,-0.23,10.36,88.40,107.91,144033.0,15289.0,162933.0,1641.55,-0.56,69.0,M-SC,7.18,220.0,0.11,1.21,22.58,OX40N,ATH,CABLES
66,SIS,-1.47,-31.69,77.51,21.26,58791.0,-35182.0,75850.0,528.00,1779.27,36.0,H-SC,5.65,168.0,-0.60,0.56,4.57,OX40N,NTT,MISC
77,TTKPRESTIG,-0.82,-32.03,47.26,0.09,32373.0,-32277.0,68500.0,770.00,59.02,31.0,M-SC,11.61,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
41,INDIGOPNTS,-0.68,-33.15,49.61,0.02,57893.0,-57863.0,116696.0,1408.00,78.01,27.0,M-SC,12.18,222.0,-1.00,0.86,1.51,OX40N,NTT,PAINTS
48,KANSAINER,-0.38,-33.37,70.31,13.47,126330.0,-89991.0,179676.0,340.00,-42.69,32.0,H-SC,13.08,162.0,-0.71,1.33,1.00,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.27,21.15,19.84,45.2,46089.0,40563.0,232305.0,1856.0,38.75,63.0,X-MC,9.12,79.0,0.88,1.72,29.39,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.00,5.53,11.50,17.67,18260.0,8326.0,158786.0,478.51,27.69,55.0,L-LC,11.02,185.0,0.46,1.18,27.47,XY25,ATH,MINING
27,FINCABLES,-0.23,10.36,88.40,107.91,144033.0,15289.0,162933.0,1641.55,-0.56,69.0,M-SC,7.18,220.0,0.11,1.21,22.58,OX40N,ATH,CABLES
50,MASFIN,-1.27,2.45,18.66,21.57,18731.0,2400.0,100380.0,397.04,-11.47,54.0,H-SC,6.73,167.0,0.13,0.74,46.48,XR,ATH,FINANCE
1,ABB,-0.78,11.64,30.39,45.57,88763.0,30461.0,292080.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.16,29.70,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.78,11.64,30.39,45.57,88763.0,30461.0,292080.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.16,29.70,AR,NTT,ELECTRICAL
50,MASFIN,-1.27,2.45,18.66,21.57,18731.0,2400.0,100380.0,397.04,-11.47,54.0,H-SC,6.73,167.0,0.13,0.74,46.48,XR,ATH,FINANCE
27,FINCABLES,-0.23,10.36,88.40,107.91,144033.0,15289.0,162933.0,1641.55,-0.56,69.0,M-SC,7.18,220.0,0.11,1.21,22.58,OX40N,ATH,CABLES
85,ZYDUSLIFE,-1.00,-2.02,38.42,35.63,78892.0,-4232.0,205342.0,1286.17,-14.86,58.0,H-MC,4.87,120.0,-0.05,1.52,14.32,AR,ATH,PHARMA
40,INDIAMART,-1.08,-7.30,123.00,106.72,140628.0,-9004.0,114332.0,4810.62,-58.20,42.0,H-SC,8.08,140.0,-0.06,0.85,14.37,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.42,-38.87,157.13,57.19,313272.0,-126756.0,199371.0,485.00,-65.01,18.0,X-SC,18.84,81.0,-0.40,1.48,0.00,XY24,NTT,FMCG
43,INFY,1.27,-11.13,51.05,34.24,141956.0,-34816.0,278072.0,1972.00,-29.97,26.0,X-LC,8.69,3.0,-0.25,2.06,2.35,X40,NTT,IT
34,HCLTECH,0.91,-10.05,37.68,23.83,81992.0,-24322.0,217602.0,1908.19,-5.71,29.0,X-LC,6.07,8.0,-0.30,1.61,4.40,X40,ATH,IT
12,BATAINDIA,0.27,-48.60,163.05,35.20,107831.0,-62536.0,66134.0,2096.00,-13.36,30.0,X-SC,22.29,91.0,-0.58,0.49,0.27,X40,NTT,FOOTWEAR
53,PGHH,0.01,-14.17,53.81,32.01,98614.0,-30256.0,183264.0,17616.87,-41.89,31.0,X-MC,9.58,57.0,-0.31,1.36,0.10,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIGI,-0.94,2.14,16.84,19.34,34419.0,4280.0,204389.0,2252.93,-14.19,53.0,X-MC,1.07,66.0,0.12,1.51,18.42,X40,ATH,INSURANCE
78,UNITDSPR,-0.48,1.54,18.94,20.77,45028.0,3594.0,237738.0,1644.00,-10.02,48.0,X-MC,1.28,64.0,0.08,1.76,9.05,X40N,NTT,BREWERIES
22,DABUR,-1.17,1.33,41.81,43.70,105100.0,3307.0,251375.0,735.00,-8.02,54.0,X-MC,1.47,70.0,0.03,1.86,16.58,XY24,BTT,FMCG
59,RELIANCE,-0.98,-10.58,40.13,25.30,72112.0,-21266.0,179697.0,1952.00,-12.73,40.0,X-LC,2.31,21.0,-0.29,1.33,20.37,XY25,BTT,REFINERIES
5,ANGELONE,-2.80,-90.19,1165.86,24.20,349175.0,-275300.0,29950.0,3033.00,-85.87,38.0,X-SC,2.50,97.0,-0.79,0.22,25.43,X40N,NTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.55,-12.07,34.28,18.07,53909.0,-21595.0,157260.0,35195.00,-25.27,36.0,X-MC,11.50,58.0,-0.40,1.16,0.0,X40,ATH,PHARMA
3,ACC,-0.84,-32.78,144.20,64.15,230648.0,-78001.0,159950.0,3906.00,-49.40,34.0,X-SC,9.49,85.0,-0.34,1.18,0.0,XY24,BTT,CEMENT
9,BAJAJHFL,-0.81,-25.60,108.96,55.47,197235.0,-62270.0,181016.0,181.50,-36.26,32.0,X-MC,17.21,60.0,-0.32,1.34,0.0,X40N,ATH,FINANCE
8,AWL,-0.42,-38.87,157.13,57.19,313272.0,-126756.0,199371.0,485.00,-65.01,18.0,X-SC,18.84,81.0,-0.40,1.48,0.0,XY24,NTT,FMCG
52,PAGEIND,-0.81,-19.02,60.71,30.14,116545.0,-45084.0,191970.0,51418.65,-40.21,33.0,X-MC,18.42,55.0,-0.39,1.42,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-2.80,-90.19,1165.86,24.20,349175.0,-275300.0,29950.0,3033.0,-85.87,38.0,X-SC,2.50,97.0,-0.79,0.22,25.43,X40N,NTT,FINANCE
54,QUESS,-0.52,-34.91,119.46,42.84,50545.0,-22695.0,42311.0,424.0,-57.58,36.0,X-SC,17.12,80.0,-0.45,0.31,1.78,XY24,NTT,MISC
58,RELAXO,-0.85,-54.52,234.90,52.31,155072.0,-79144.0,66016.0,1176.0,-51.60,36.0,X-SC,14.79,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.27,-48.60,163.05,35.20,107831.0,-62536.0,66134.0,2096.0,-13.36,30.0,X-SC,22.29,91.0,-0.58,0.49,0.27,X40,NTT,FOOTWEAR
51,MEDANTA,-1.23,-4.20,29.51,24.08,35891.0,-5326.0,121624.0,1486.0,-4.88,51.0,X-SC,7.96,87.0,-0.15,0.90,11.40,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-1.07,-17.06,43.54,19.06,122848.0,-58016.0,282150.0,452.00,-54.31,40.0,X-LC,17.19,1.0,-0.47,2.09,2.16,X40,NTT,FMCG
73,TCS,0.22,-23.95,62.49,23.58,205606.0,-103607.0,329022.0,4311.59,-34.12,32.0,X-LC,9.12,2.0,-0.50,2.44,3.10,X40,ATH,IT
43,INFY,1.27,-11.13,51.05,34.24,141956.0,-34816.0,278072.0,1972.00,-29.97,26.0,X-LC,8.69,3.0,-0.25,2.06,2.35,X40,NTT,IT
75,TMPV,-0.52,-13.44,54.04,33.33,152392.0,-43802.0,281998.0,600.00,-81.44,64.0,X-LC,3.96,4.0,-0.29,2.09,15.29,XY24,NTT,AUTO
34,HCLTECH,0.91,-10.05,37.68,23.83,81992.0,-24322.0,217602.0,1908.19,-5.71,29.0,X-LC,6.07,8.0,-0.30,1.61,4.40,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,-0.46,-29.11,87.47,32.89,47871.0,-22478.0,54729.0,1800.00,-357.70,65.0,L-MC,12.11,258.0,-0.47,0.41,50.72,XR,NTT,BANKS
67,SONACOMS,-1.48,-7.05,49.64,39.09,46675.0,-7132.0,94027.0,804.02,-26.41,61.0,M-MC,10.85,193.0,-0.15,0.70,33.02,AR,ATH,AUTO
50,MASFIN,-1.27,2.45,18.66,21.57,18731.0,2400.0,100380.0,397.04,-11.47,54.0,H-SC,6.73,167.0,0.13,0.74,46.48,XR,ATH,FINANCE
19,COALINDIA,-1.00,5.53,11.50,17.67,18260.0,8326.0,158786.0,478.51,27.69,55.0,L-LC,11.02,185.0,0.46,1.18,27.47,XY25,ATH,MINING
7,ATULAUTO,0.46,-12.88,67.10,45.57,115234.0,-25398.0,171734.0,844.00,3721.62,61.0,M-SC,6.73,248.0,-0.22,1.27,29.96,XY24,NTT,AUTO


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,-0.58,-52.37,205.91,45.70,173401.0,-92603.0,84212.0,26.40,-12.47,61.0,M-SC,18.19,204.0,-0.53,0.62,38.75,XY24,NTT,TRAVEL
7,ATULAUTO,0.46,-12.88,67.10,45.57,115234.0,-25398.0,171734.0,844.00,3721.62,61.0,M-SC,6.73,248.0,-0.22,1.27,29.96,XY24,NTT,AUTO
1,ABB,-0.78,11.64,30.39,45.57,88763.0,30461.0,292080.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.16,29.70,AR,NTT,ELECTRICAL
27,FINCABLES,-0.23,10.36,88.40,107.91,144033.0,15289.0,162933.0,1641.55,-0.56,69.0,M-SC,7.18,220.0,0.11,1.21,22.58,OX40N,ATH,CABLES
11,BANDHANBNK,-3.69,-13.78,123.18,92.42,295839.0,-38391.0,240168.0,400.00,106.18,67.0,H-SC,8.04,174.0,-0.13,1.78,32.30,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.82
1,25,45.85
2,50,77.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.94
MC    30.81
LC    25.21
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.52
X40      24.24
X40N     12.93
AR        9.32
XY25      9.10
XR        8.69
OX40N     7.31
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.17
H-SC    22.70
X-LC    21.77
M-SC    10.98
X-SC     9.54
H-MC     5.09
M-MC     2.14
H-LC     1.22
L-LC     1.18
M-LC     1.04
L-SC     0.72
L-MC     0.41
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.96,-14.57,53.31
IT,11.60,-40.04,110.23
FINANCE,11.24,-37.53,91.65
MISC,6.95,-38.49,95.14
ELECTRICAL,6.49,-7.00,46.04
PAINTS,5.10,-33.73,53.60
INSURANCE,5.00,-0.25,34.55
PHARMA,4.22,-2.98,35.39
AUTO,4.06,-13.94,57.38


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3410296.0,21
X40,1446085.0,15
X40N,1376092.0,10
XR,1317724.0,12
AR,1312341.0,10
OX40N,804921.0,10
XY25,389561.0,6
SR,304499.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3725988.0,24
X-SC,1514584.0,10
M-SC,1435770.0,14
X-MC,1384027.0,15
X-LC,1333211.0,12
H-MC,390614.0,3
L-SC,193145.0,2
M-LC,151686.0,1
M-MC,98047.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1261943.0      6
           AR         954285.0      5
           XR         841528.0      7
M-SC       XY24       835678.0      6
X-LC       X40        709881.0      6
X-SC       X40N       699459.0      4
           XY24       630356.0      4
X-MC       X40        551435.0      7
           X40N       446806.0      4
H-SC       OX40N      363733.0      4
M-SC       OX40N      343138.0      5
H-SC       SR         304499.0      2
X-LC       XY24       264362.0      2
           X40N       229827.0      2
H-MC       XY24       222959.0      1
X-MC       XY24       194998.0      2
           XY25       190788.0      2
X-SC       X40        184769.0      2
M-SC       XR         181544.0      2
H-MC       AR         167655.0      2
M-LC       XR         151686.0      1
X-LC       XY25       129141.0      2
L-SC       OX40N       98050.0      1
           XR          95095.0      1
M-SC       AR          75410.0      1
H-LC       AR          68316.0      1
M-MC       XY25        51372.0      1
L-MC       XR          47871.0      1
M-MC       AR          46675.0      1
L-LC       XY25        18260.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
